# Solutions of exercises

Exercises are available in a few notebooks. Below are solutions to the exercises.
How to use the notebook:

* **DO NOT RUN THE NOTEBOOK**
* **Copy the code snippet(s) and run the snippet(s) in the relevant notebook**


## Exercises from notebook idr0062_prediction

### Exercise 1 

In [ ]:
# Question 1: Read the geojson file:
import json
with open(geojson_file) as f:
    d = json.load(f)
    values = d['features']

In [ ]:
# Question 2: Overlay the shapes from a specific plane on the image
z = 5
ax = plt.subplot()
plt.imshow(data[0, 0, z, :, :], cmap='gray')
for v in values:
    if v["properties"]["z"] == z:
        label_geojson = numpy.array(v["geometry"]["coordinates"])
        p = label_geojson.squeeze()
        ax.plot(p[:, 0], p[:, 1], linewidth=v["properties"]["stroke-width"], color="red")
plt.show()

### Exercise 2 

In order to save the StarDist labels to an OMERO.server with write access, you will first need to log in and change the image ID.

In [1]:
from omero.gateway import BlitzGateway
from getpass import getpass
# To be replaced by the desired host
HOST = 'wss://workshop.openmicroscopy.org/omero-ws'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()
print(conn.connect())

False


In [ ]:
# Question 1: Convert the StarDist labels into OMERO polygons
import omero
from omero.rtypes import rint, rstring

image_id = REPLACE_WITH_IMAGE_ID

image = omero.model.ImageI(image_id, False)
rois = []
for i in range(len(img)):
    new_labels, details = model_versatile.predict_instances(img[i])
    # Convert into Polygon
    for obj_id, region in enumerate(details['coord']):
        roi = omero.model.RoiI()
        roi.setName(rstring("Object %s" % obj_id))
        polygon = omero.model.PolygonI()
        x = region[1]
        y = region[0]
        points = " ".join(
            ["{},{}".format(x[j], y[j]) for j in range(len(x))])
        polygon.setPoints(rstring(points))
        polygon.theZ = rint(i)
        polygon.theC = rint(0)
        roi.addShape(polygon)
        # Link the ROI and the image
        roi.setImage(image)
        rois.append(roi)                    

In [ ]:
# Question 2: Save the ROI
rois = conn.getUpdateService().saveAndReturnArray(rois)